# Imports

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import StringType

session = get_active_session()

# 1. Transcribe Audio Files

Easily transcribe audio files from stages using Snowflake's built-in [AI_TRANSCRIBE](https://docs.snowflake.com/en/sql-reference/functions/ai_transcribe) function.

In [ ]:
-- Show files in stage
SELECT * FROM DIRECTORY(@AUDIO_FILES);

In [ ]:
CREATE OR REPLACE TABLE RAW_TRANSCRIPTIONS AS
WITH CALL_CENTER_TRANSCRIPTIONS AS (
    SELECT
        -- Extract metadata from path
        split(RELATIVE_PATH,'/')[1]::DATE AS DATE,
        split(RELATIVE_PATH,'/')[2]::string AS AUDIO_FILE,
        RELATIVE_PATH AS AUDIOFILE_RELATIVE_PATH,
        
        -- Transcribe audio file
        AI_TRANSCRIBE(TO_FILE('@AUDIO_FILES', RELATIVE_PATH)) AS TRANSCRIPTION,
        TRANSCRIPTION['text']::VARCHAR AS TRANSCRIPTION_RAW_TEXT,
        TRANSCRIPTION['audio_duration']::FLOAT AS TRANSCRIPTION_DURATION_SECONDS,
        GET_PRESIGNED_URL('@AUDIO_FILES', RELATIVE_PATH) AS PRESIGNED_URL
    FROM DIRECTORY('@AUDIO_FILES')
)
SELECT 
    DATE,
    AUDIO_FILE,
    AUDIOFILE_RELATIVE_PATH,
    TRANSCRIPTION_RAW_TEXT,
    TRANSCRIPTION_DURATION_SECONDS,
    PRESIGNED_URL
FROM
    CALL_CENTER_TRANSCRIPTIONS;

SELECT * FROM RAW_TRANSCRIPTIONS

# 2. LLMs for Structuring and Analyzing Calls

We use Snowflake [AI_COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/ai_complete-single-string) function in combination with [PROMPT](https://docs.snowflake.com/en/sql-reference/functions/prompt) to use templates for prompting LLMs.  
With such templates we can easily structure the transcription and extract key information in a structured manner.

In addition we use [SUMMARIZE](https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex) to create short summaries for all calls and [AI_SENTIMENT](https://docs.snowflake.com/en/sql-reference/functions/ai_sentiment) to analyze each call's sentiment.

In [ ]:
CREATE OR REPLACE TABLE ANALYZED_TRANSCRIPTIONS AS
WITH TRANSCRIPTIONS_INSIGHTS AS (
    SELECT
        -- Structure the conversation
        PROMPT('
            Convert this transcript to a structured conversation format:
            Representative: [Name]
            Customer: [Name]
            
            [Conversation lines...]
            
            <transcript>{0}</transcript>
            ',
            TRANSCRIPTION_RAW_TEXT
            ) AS PROMPT_STRUCTURE_CONVERSATION,
        AI_COMPLETE('claude-3-5-sonnet', PROMPT_STRUCTURE_CONVERSATION) AS CONVERSATION_STRUCTURED,
        
        -- Create a conversation summary
        SNOWFLAKE.CORTEX.SUMMARIZE(CONVERSATION_STRUCTURED) AS CONVERSATION_SUMMARY,

        -- Create a conversation sentiment
        AI_SENTIMENT(CONVERSATION_STRUCTURED) AS CONVERSATION_SENTIMENT,

        -- Extract details from conversation
        PROMPT('
        You are analyzing call center transcripts to extract specific information. 
        Extract the following data from the transcript and return ONLY a valid JSON response with no additional text.
        "representative": "string",
        "customer": "string", 
        "claim_number": "string",
        "policy_number": "string",
        "call_intent": "string",
        "call_to_action": "string",
        "issue": "string",
        "resolution": "string",
        "next_steps": "string",
        "mode_of_update": "string",
        "purpose_of_call": "string",
        "response_mode": "string",
        "first_call_resolution": number,
        "call_quality": "string",
        "net_promoter_score": integer
        
        FIELD SPECIFICATIONS:
        
        Representative:
        - representative name

        Customer:
        - customer name
        
        Claim Number:
        - Format: "CLM" + digits (e.g., "CLM123456") OR "NotFound"  
        - Convert variations: "CLM 456789" → "CLM456789", "CLM 789-012" → "CLM789012"
        - If no valid number found: "NotFound"

        Policy Number:
        - Format: "POL" + digits (e.g., "POL123456") OR "NotFound"
        - Convert variations: "PAUL 348743" → "POL348743", "POL 123-456" → "POL123456", "POLL123456" → "POL123456"
        - If no valid number found: "NotFound"
        
        Call Intent:
        - Maximum 2 words
        - Cannot be "collision with animal"

        Call to Action:
        - brief action description

        Isse:
        - brief issue description

        Resolution:
        - brief resolution

        Next Steps:
        - brief next steps

        Mode of Update:
        - Values: "Email", "Phone", or "Portal"

        Purpose of Call:
        - brief purpose

        Response Mode:
        - Follow-up method
        
        First Call Resolution:
        - Score between 0.01-0.99
        - 0.01-0.30: Issue not resolved, requires follow-up
        - 0.31-0.70: Partial resolution, some follow-up needed
        - 0.71-0.99: Issue fully resolved in single call
        
        Call Quality:
        - Values: "Good", "Average", or "Poor"
        - Based on communication skills, protocol adherence, customer handling
        
        Net Promoter Score:
        - Integer between 10-100
        - Consider resolution time and representative skill
        
        TRANSCRIPT TO ANALYZE:
        {0}
        ', 
        CONVERSATION_STRUCTURED) AS PROMPT_EXTRACT_CONVERSATION_DETAILS,
        AI_COMPLETE(
            model => 'claude-3-5-sonnet',
            prompt => PROMPT_EXTRACT_CONVERSATION_DETAILS,
            response_format => {
                'type':'json',
                'schema':{
                    'type':'object',
                    'properties':{
                        'representative':{'type':'string'},
                        'customer':{'type':'string'},
                        'claim_number':{'type':'string'},
                        'policy_number':{'type':'string'},
                        'call_intent':{'type':'string'},
                        'call_to_action':{'type':'string'},
                        'issue':{'type':'string'},
                        'resolution':{'type':'string'},
                        'next_steps':{'type':'string'},
                        'mode_of_update':{'type':'string'},
                        'purpose_of_call':{'type':'string'},
                        'response_mode':{'type':'string'},
                        'first_call_resolution':{'type':'number'},
                        'call_quality':{'type':'string'},
                        'net_promoter_score':{'type':'integer'}
                    }
                }
            }
        ) AS _CONVERSATION_DETAILS,
    try_parse_json(_CONVERSATION_DETAILS) AS CONVERSATION_DETAILS,

    -- Other columns
    DATE,
    AUDIO_FILE,
    AUDIOFILE_RELATIVE_PATH,
    TRANSCRIPTION_RAW_TEXT,
    TRANSCRIPTION_DURATION_SECONDS,
    PRESIGNED_URL
    
    FROM
        RAW_TRANSCRIPTIONS
)
SELECT 
    DATE,
    AUDIO_FILE,
    AUDIOFILE_RELATIVE_PATH,
    TRANSCRIPTION_RAW_TEXT,
    TRANSCRIPTION_DURATION_SECONDS,
    CONVERSATION_STRUCTURED,
    CONVERSATION_SUMMARY,
    CONVERSATION_SENTIMENT,
    CONVERSATION_DETAILS,
    PRESIGNED_URL
FROM TRANSCRIPTIONS_INSIGHTS;

SELECT * FROM ANALYZED_TRANSCRIPTIONS;

# 3. Create a BI-Ready Table for Call Center Analysis

Let's create a beautiful structured data for our final Call Center Analysis App.

In [ ]:
CREATE OR REPLACE TABLE ANALYZED_TRANSCRIPTIONS_APP AS 
SELECT 
    DATE,
    AUDIO_FILE,
    AUDIOFILE_RELATIVE_PATH,
    TRANSCRIPTION_DURATION_SECONDS,
    CONVERSATION_DETAILS['call_to_action']::TEXT as CALL_TO_ACTION,
    CONVERSATION_DETAILS['claim_number']::TEXT as CLAIM_NUMBER,
    CONVERSATION_DETAILS['customer']::TEXT as CUSTOMER,
    CONVERSATION_DETAILS['call_intent']::TEXT as CALL_INTENT,
    CONVERSATION_DETAILS['issue']::TEXT as ISSUE,
    CONVERSATION_DETAILS['policy_number']::TEXT as POLICY_NUMBER,
    CONVERSATION_DETAILS['purpose_of_call']::TEXT as PURPOSE_OF_CALL,
    CONVERSATION_DETAILS['representative']::TEXT as REPRESENTATIVE,
    CONVERSATION_DETAILS['resolution']::TEXT as RESOLUTION,
    CONVERSATION_DETAILS['response_mode']::TEXT as RESPONSE_MODE,
    CONVERSATION_DETAILS['mode_of_update']::TEXT as MODE_OF_UPDATE,
    CONVERSATION_DETAILS['next_steps']::TEXT as NEXT_STEPS,
    IFF(CONVERSATION_SENTIMENT['categories'][0]['sentiment']='positive','Positive','Negative') as CONVERSATION_SENTIMENT,
    IFF(CONVERSATION_DETAILS['first_call_resolution'] > 0.5,'Yes','No') as FIRST_CALL_RESOLUTION,
    CONVERSATION_SUMMARY,
    CONVERSATION_STRUCTURED
FROM 
    ANALYZED_TRANSCRIPTIONS;

SELECT * FROM ANALYZED_TRANSCRIPTIONS_APP;

# 4. Automate the entire process in a Stored Procedure

We don't want to rerun this notebook everytime a new file is uploaded, hence we create a Stored Procedure for it that we can call for new files.  
The same procedure can be integrated into the Call Center App to allow ad-hoc analysis of uploaded call recordings.

In [ ]:
@F.sproc(name="process_audio_file", return_type=StringType(), input_types=[StringType()], packages=["snowflake-snowpark-python"], stage_location='@UDF', is_permanent=True, replace=True)
def process_audio_file(session, audio_file_relative_path):
    session.sql(f"""
    -- Transcribe new file and add it to raw transcriptions
    INSERT INTO RAW_TRANSCRIPTIONS
    WITH CALL_CENTER_TRANSCRIPTIONS AS (
        SELECT
            -- Extract metadata from path
            split(RELATIVE_PATH,'/')[1]::DATE AS DATE,
            split(RELATIVE_PATH,'/')[2]::string AS AUDIO_FILE,
            RELATIVE_PATH AS AUDIOFILE_RELATIVE_PATH,
            
            -- Transcribe audio file
            AI_TRANSCRIBE(TO_FILE('@AUDIO_FILES', RELATIVE_PATH)) AS TRANSCRIPTION,
            TRANSCRIPTION['text']::VARCHAR AS TRANSCRIPTION_RAW_TEXT,
            TRANSCRIPTION['audio_duration']::FLOAT AS TRANSCRIPTION_DURATION_SECONDS,
            GET_PRESIGNED_URL('@AUDIO_FILES', RELATIVE_PATH) AS PRESIGNED_URL
        FROM DIRECTORY('@AUDIO_FILES')
        WHERE RELATIVE_PATH = '{audio_file_relative_path}'
    )
    SELECT 
        DATE,
        AUDIO_FILE,
        AUDIOFILE_RELATIVE_PATH,
        TRANSCRIPTION_RAW_TEXT,
        TRANSCRIPTION_DURATION_SECONDS,
        PRESIGNED_URL
    FROM
        CALL_CENTER_TRANSCRIPTIONS
    """).collect()

    session.sql(f"""
    INSERT INTO ANALYZED_TRANSCRIPTIONS
    WITH TRANSCRIPTIONS_INSIGHTS AS (
        SELECT
            -- Structure the conversation
            PROMPT('
                Convert this transcript to a structured conversation format:
                Representative: [Name]
                Customer: [Name]
                
                [Conversation lines...]
                
                <transcript>{{0}}</transcript>
                ',
                TRANSCRIPTION_RAW_TEXT
                ) AS PROMPT_STRUCTURE_CONVERSATION,
            AI_COMPLETE('claude-3-5-sonnet', PROMPT_STRUCTURE_CONVERSATION) AS CONVERSATION_STRUCTURED,
            
            -- Create a conversation summary
            SNOWFLAKE.CORTEX.SUMMARIZE(CONVERSATION_STRUCTURED) AS CONVERSATION_SUMMARY,
    
            -- Create a conversation sentiment
            AI_SENTIMENT(CONVERSATION_STRUCTURED) AS CONVERSATION_SENTIMENT,
    
            -- Extract details from conversation
            PROMPT('
            You are analyzing call center transcripts to extract specific information. 
            Extract the following data from the transcript and return ONLY a valid JSON response with no additional text.
            "representative": "string",
            "customer": "string", 
            "claim_number": "string",
            "policy_number": "string",
            "call_intent": "string",
            "call_to_action": "string",
            "issue": "string",
            "resolution": "string",
            "next_steps": "string",
            "mode_of_update": "string",
            "purpose_of_call": "string",
            "response_mode": "string",
            "first_call_resolution": number,
            "call_quality": "string",
            "net_promoter_score": integer
            
            FIELD SPECIFICATIONS:
            
            Representative:
            - representative name
    
            Customer:
            - customer name
            
            Claim Number:
            - Format: "CLM" + digits (e.g., "CLM123456") OR "NotFound"  
            - Convert variations: "CLM 456789" → "CLM456789", "CLM 789-012" → "CLM789012"
            - If no valid number found: "NotFound"
    
            Policy Number:
            - Format: "POL" + digits (e.g., "POL123456") OR "NotFound"
            - Convert variations: "PAUL 348743" → "POL348743", "POL 123-456" → "POL123456", "POLL123456" → "POL123456"
            - If no valid number found: "NotFound"
            
            Call Intent:
            - Maximum 2 words
            - Cannot be "collision with animal"
    
            Call to Action:
            - brief action description
    
            Isse:
            - brief issue description
    
            Resolution:
            - brief resolution
    
            Next Steps:
            - brief next steps
    
            Mode of Update:
            - Values: "Email", "Phone", or "Portal"
    
            Purpose of Call:
            - brief purpose
    
            Response Mode:
            - Follow-up method
            
            First Call Resolution:
            - Score between 0.01-0.99
            - 0.01-0.30: Issue not resolved, requires follow-up
            - 0.31-0.70: Partial resolution, some follow-up needed
            - 0.71-0.99: Issue fully resolved in single call
            
            Call Quality:
            - Values: "Good", "Average", or "Poor"
            - Based on communication skills, protocol adherence, customer handling
            
            Net Promoter Score:
            - Integer between 10-100
            - Consider resolution time and representative skill
            
            TRANSCRIPT TO ANALYZE:
            {{0}}
            ', 
            CONVERSATION_STRUCTURED) AS PROMPT_EXTRACT_CONVERSATION_DETAILS,
            AI_COMPLETE(
                model => 'claude-3-5-sonnet',
                prompt => PROMPT_EXTRACT_CONVERSATION_DETAILS,
                response_format => {{
                    'type':'json',
                    'schema':{{
                        'type':'object',
                        'properties':{{
                            'representative':{{'type':'string'}},
                            'customer':{{'type':'string'}},
                            'claim_number':{{'type':'string'}},
                            'policy_number':{{'type':'string'}},
                            'call_intent':{{'type':'string'}},
                            'call_to_action':{{'type':'string'}},
                            'issue':{{'type':'string'}},
                            'resolution':{{'type':'string'}},
                            'next_steps':{{'type':'string'}},
                            'mode_of_update':{{'type':'string'}},
                            'purpose_of_call':{{'type':'string'}},
                            'response_mode':{{'type':'string'}},
                            'first_call_resolution':{{'type':'number'}},
                            'call_quality':{{'type':'string'}},
                            'net_promoter_score':{{'type':'integer'}}
                        }}
                    }}
                }}
            ) AS _CONVERSATION_DETAILS,
        try_parse_json(_CONVERSATION_DETAILS) AS CONVERSATION_DETAILS,
    
        -- Other columns
        DATE,
        AUDIO_FILE,
        AUDIOFILE_RELATIVE_PATH,
        TRANSCRIPTION_RAW_TEXT,
        TRANSCRIPTION_DURATION_SECONDS,
        PRESIGNED_URL
        
        FROM
            RAW_TRANSCRIPTIONS
        WHERE
            AUDIOFILE_RELATIVE_PATH = '{audio_file_relative_path}'
    )
    SELECT 
        DATE,
        AUDIO_FILE,
        AUDIOFILE_RELATIVE_PATH,
        TRANSCRIPTION_RAW_TEXT,
        TRANSCRIPTION_DURATION_SECONDS,
        CONVERSATION_STRUCTURED,
        CONVERSATION_SUMMARY,
        CONVERSATION_SENTIMENT,
        CONVERSATION_DETAILS,
        PRESIGNED_URL
    FROM TRANSCRIPTIONS_INSIGHTS;
    """).collect()

    session.sql(f"""
    INSERT INTO ANALYZED_TRANSCRIPTIONS_APP
    SELECT 
        DATE,
        AUDIO_FILE,
        AUDIOFILE_RELATIVE_PATH,
        TRANSCRIPTION_DURATION_SECONDS,
        CONVERSATION_DETAILS['call_to_action']::TEXT as CALL_TO_ACTION,
        CONVERSATION_DETAILS['claim_number']::TEXT as CLAIM_NUMBER,
        CONVERSATION_DETAILS['customer']::TEXT as CUSTOMER,
        CONVERSATION_DETAILS['call_intent']::TEXT as CALL_INTENT,
        CONVERSATION_DETAILS['issue']::TEXT as ISSUE,
        CONVERSATION_DETAILS['policy_number']::TEXT as POLICY_NUMBER,
        CONVERSATION_DETAILS['purpose_of_call']::TEXT as PURPOSE_OF_CALL,
        CONVERSATION_DETAILS['representative']::TEXT as REPRESENTATIVE,
        CONVERSATION_DETAILS['resolution']::TEXT as RESOLUTION,
        CONVERSATION_DETAILS['response_mode']::TEXT as RESPONSE_MODE,
        CONVERSATION_DETAILS['mode_of_update']::TEXT as MODE_OF_UPDATE,
        CONVERSATION_DETAILS['next_steps']::TEXT as NEXT_STEPS,
        IFF(CONVERSATION_SENTIMENT['categories'][0]['sentiment']='positive','Positive','Negative') as CONVERSATION_SENTIMENT,
        IFF(CONVERSATION_DETAILS['first_call_resolution'] > 0.5,'Yes','No') as FIRST_CALL_RESOLUTION,
        CONVERSATION_SUMMARY,
        CONVERSATION_STRUCTURED
    FROM 
        ANALYZED_TRANSCRIPTIONS
    WHERE
        AUDIOFILE_RELATIVE_PATH = '{audio_file_relative_path}'
    """).collect()

    return f'Successfully processed file {audio_file_relative_path}.'

In [ ]:
-- CALL process_audio_file('audio_files/2024-11-11/audiofile23.mp3');